<a href="https://colab.research.google.com/github/Puru35/Capastone-1/blob/master/Data%20Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before we start,  **please read the description of this capstone project.**

Link to the Original Data Sets: https://archive.ics.uci.edu/ml/datasets/Student+Performance

I have split my project into 2 notebooks, one for data cleaning and preprocessing, and the other for model creation, training,  and testing. The reason for this, is because I used 2 different ways to clean my data set, and both of them when plugged to my model, yielded different results. Having said that, let's start.

# Importing Necessary Libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Importing the Data Set.


In [0]:
data=pd.read_csv("student-mat.csv")

In [0]:
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


Checking if there are any missing values in the data set.

In [0]:
data.isnull().any()

school        False
sex           False
age           False
address       False
famsize       False
Pstatus       False
Medu          False
Fedu          False
Mjob          False
Fjob          False
reason        False
guardian      False
traveltime    False
studytime     False
failures      False
schoolsup     False
famsup        False
paid          False
activities    False
nursery       False
higher        False
internet      False
romantic      False
famrel        False
freetime      False
goout         False
Dalc          False
Walc          False
health        False
absences      False
G1            False
G2            False
G3            False
dtype: bool

As discussed earlier, I have used two ways to clean my data. The first one, is a function that takes the entire data set and an empty data set as it's parameters. If there are any string values in any of the attributes, it uses label binarizer the split the column into multiple columns based on the number of unique values present in that specific attribute.

# Data cleaning and preprocessing

##1. Function that takes any data set and cleans it.

In [0]:
def clean(data,df):
    from sklearn.preprocessing import LabelBinarizer
    binarizer = LabelBinarizer()
    column_names = list(data.columns)
    for i in range(len(data.columns)):
        if isinstance(data[column_names[i]][0], str) and data[column_names[i]].nunique()>2:
            column =data[column_names[i]].values
            x = binarizer.fit_transform(column)
            temp_df = pd.DataFrame(x, columns = binarizer.classes_)
            df = pd.concat([df,temp_df],axis = 1, ignore_index = False)
        elif isinstance(data[column_names[i]][0], str) and data[column_names[i]].nunique()==2:
            if data[column_names[i]][0] == "no" or data[column_names[i]][0] == "yes":
                templist = []
                for r in range(len(data[column_names[i]])):
                    if data[column_names[i]][r] == "no":
                        templist.append(0)
                    else:
                        templist.append(1)
                temp_df = pd.DataFrame(templist)
                temp_df = temp_df.rename(columns={0 : data.columns[i]})   
            else:
                column =data[column_names[i]].values
                x = binarizer.fit_transform(column)
                temp_df = pd.DataFrame(x)
                temp_df = temp_df.rename(columns={0 : binarizer.classes_[0]})
            df = pd.concat([df,temp_df],axis = 1, ignore_index = False)
        else:
            df = pd.concat([df,data[column_names[i]]],axis = 1,ignore_index = False)
    return df

In [0]:
df = pd.DataFrame()
df = clean(data,df)

###New Data set with only numeric values.

In [0]:
df.head()

,GP,F,age,R,GT3,A,Medu,Fedu,at_home,health,other,services,teacher,at_home,health,other,services,teacher,course,home,other,reputation,father,mother,other,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,0,18,1,0,0,4,4,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,2,2,0,1,0,0,0,1,1,0,0,4,3,4,1,1,3,6,5,6,6
1,0,0,17,1,0,1,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,2,0,0,1,0,0,0,1,1,0,5,3,3,1,1,3,4,5,5,6
2,0,0,15,1,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,2,3,1,0,1,0,1,1,1,0,4,3,2,2,3,3,10,7,8,10
3,0,0,15,1,0,1,4,2,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,3,0,0,1,1,1,1,1,1,1,3,2,2,1,1,5,2,15,14,15
4,0,0,16,1,0,1,3,3,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,2,0,0,1,1,0,1,1,0,0,4,3,2,1,2,5,4,6,10,10


We now see that the old data set hs been converted into a data set that does not contain any string values, only numeric.

####Total attributes of old Data set.

In [0]:
len(data.columns)

33

####Total attributes of new Data set

In [0]:
len(df.columns)

46

The problem with the first way, even though as we see later on, does give a better test accuracy, is that it creates a very sparse data set with a lot of zeros, which affects the correlation of each attribute. The second way given below, makes changes in the actual data set itself, and does not split the columns, rather it assigns numbers to each unique value of the attribute. So the number of columns remain the same. 

Even though this does cover most of the problems posed ny the first method of cleaning, the one major drawback, is that when numbers are assigned to each value, a bias is introduced, wherein the model may consider one value to be greater than the other, whereas it is not actually the case.

##2. Using cat.codes to assign values in the data set.

In [0]:
dff = data

In [0]:
for i in range (len(data.columns)):
    dff[dff.columns[i]] = dff[data.columns[i]].astype("category").cat.codes

In [0]:
dff.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,0,3,1,0,0,4,4,0,4,0,1,1,1,0,1,0,0,0,1,1,0,0,3,2,3,0,0,2,6,2,3,3
1,0,0,2,1,0,1,1,1,0,2,0,0,0,1,0,0,1,0,0,0,1,1,0,4,2,2,0,0,2,4,2,2,3
2,0,0,0,1,1,1,1,1,0,2,2,1,0,1,3,1,0,1,0,1,1,1,0,3,2,1,1,2,2,10,4,5,7
3,0,0,0,1,0,1,4,2,1,3,1,1,0,2,0,0,1,1,1,1,1,1,1,2,1,1,0,0,4,2,12,11,12
4,0,0,1,1,0,1,3,3,2,2,1,0,0,1,0,0,1,1,0,1,1,0,0,3,2,1,0,1,4,4,3,7,7


In [0]:
len(dff.columns)

33

As stated in the description, we were given 2 similar data sets. Since we have done two different methods of cleaning the train data set, we must do the same for the test data set as well. The explanation remains the same. 

##Test Data Set

In [0]:
data2 = pd.read_csv("student-por.csv")

In [0]:
data2.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [0]:
data2.isnull().any()

school        False
sex           False
age           False
address       False
famsize       False
Pstatus       False
Medu          False
Fedu          False
Mjob          False
Fjob          False
reason        False
guardian      False
traveltime    False
studytime     False
failures      False
schoolsup     False
famsup        False
paid          False
activities    False
nursery       False
higher        False
internet      False
romantic      False
famrel        False
freetime      False
goout         False
Dalc          False
Walc          False
health        False
absences      False
G1            False
G2            False
G3            False
dtype: bool

###Method 1

In [0]:
df2 = pd.DataFrame()
df2 = clean(data2,df2)

In [0]:
df2.head()

,GP,F,age,R,GT3,A,Medu,Fedu,at_home,health,other,services,teacher,at_home,health,other,services,teacher,course,home,other,reputation,father,mother,other,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,0,18,1,0,0,4,4,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,2,2,0,1,0,0,0,1,1,0,0,4,3,4,1,1,3,4,0,11,11
1,0,0,17,1,0,1,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,2,0,0,1,0,0,0,1,1,0,5,3,3,1,1,3,2,9,11,11
2,0,0,15,1,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,2,0,1,0,0,0,1,1,1,0,4,3,2,2,3,3,6,12,13,12
3,0,0,15,1,0,1,4,2,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,3,0,0,1,0,1,1,1,1,1,3,2,2,1,1,5,0,14,14,14
4,0,0,16,1,0,1,3,3,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,2,0,0,1,0,0,1,1,0,0,4,3,2,1,2,5,0,11,13,13


In [0]:
len(data2.columns)

33

In [0]:
len(df2.columns)

46

###Method 2

In [0]:
dff2 = data2

In [0]:
for i in range (len(data2.columns)):
    dff2[dff2.columns[i]] = dff2[data2.columns[i]].astype("category").cat.codes

In [0]:
dff2.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,0,3,1,0,0,4,4,0,4,0,1,1,1,0,1,0,0,0,1,1,0,0,3,2,3,0,0,2,4,0,7,8
1,0,0,2,1,0,1,1,1,0,2,0,0,0,1,0,0,1,0,0,0,1,1,0,4,2,2,0,0,2,2,6,7,8
2,0,0,0,1,1,1,1,1,0,2,2,1,0,1,0,1,0,0,0,1,1,1,0,3,2,1,1,2,2,6,9,9,9
3,0,0,0,1,0,1,4,2,1,3,1,1,0,2,0,0,1,0,1,1,1,1,1,2,1,1,0,0,4,0,11,10,11
4,0,0,1,1,0,1,3,3,2,2,1,0,0,1,0,0,1,0,0,1,1,0,0,3,2,1,0,1,4,0,8,9,10


In [0]:
len(dff2.columns)

33

After cleaning the train and test data set in both methods, we end up with 4 new data sets, which we will then convert to csv format and save, so that we can use them in the model training part of the project. 

In [0]:
df.to_csv("Train 1.csv")
dff.to_csv("Train 2.csv")
df2.to_csv("Test 1.csv")
dff2.to_csv("Test 2.csv")